<a href="https://colab.research.google.com/github/julianovale/BB_Evolution/blob/main/0017_AluraBB_RedesNeurais_TreinandoRedeNeural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


# Dataset: Classificação de vinhos

3 classes, 13 features e 178 amostras


In [4]:
from sklearn import datasets
wine = datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']


In [5]:
target # numéricos

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [6]:
data # numéricos

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [7]:
class WineClassifier(nn.Module):

  def __init__(self, input_size, hidden_size, out_size):
    super(WineClassifier, self).__init__()

    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.out = nn.Linear(hidden_size, out_size)
    self.softmax = nn.Softmax()

  def forward(self, X):

    feature = self.relu(self.hidden(X))
    output = self.softmax(self.out(feature))

    return output

input_size = data.shape[1]
hidden_size = 32
out_size = len(wine.target_names)

net = WineClassifier(input_size, hidden_size, out_size).to(device) # cast na GPU

In [8]:
net

WineClassifier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)

In [9]:
criterion = nn.CrossEntropyLoss().to(device) # como é um problema de classificação = entropia cruzada

In [13]:
Xtns = torch.from_numpy(data).float()
Ytns = torch.from_numpy(target)

Xtns = Xtns.to(device)
Ytns = Ytns.to(device)


print(Xtns.dtype, Ytns.dtype)

torch.float32 torch.int64


In [16]:
pred = net(Xtns)

print(Xtns.shape)
print(pred.shape, Ytns.shape)

torch.Size([178, 13])
torch.Size([178, 3]) torch.Size([178])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [18]:
print(pred[0].data, Ytns[0].data) # a predição é uma distribuição de probabilidade e o rótulo é um único valor

tensor([8.5607e-02, 5.7574e-33, 9.1439e-01], device='cuda:0') tensor(0, device='cuda:0')


In [22]:
loss = criterion(pred, Ytns) # verificar minha função de perda (função objetivo): todos as amostras
print(loss) # perda média (erro médio) ==> A LOSS é sempre um escalar que indica o erro naquele conjunto

tensor(1.2733, device='cuda:0', grad_fn=<NllLossBackward>)


In [24]:
loss = criterion(pred[0].unsqueeze(0), Ytns[0].unsqueeze(0)) # verificar minha função de perda (função objetivo): uma única amostra
print(loss) # perda média (erro médio) ==> A LOSS é sempre um escalar que indica o erro naquele conjunto

tensor(1.4371, device='cuda:0', grad_fn=<NllLossBackward>)


In [25]:
loss = criterion(pred[:30], Ytns[:30]) # verificar minha função de perda (função objetivo): uma intervalo de amostras
print(loss) # perda média (erro médio) ==> A LOSS é sempre um escalar que indica o erro naquele conjunto

tensor(1.4377, device='cuda:0', grad_fn=<NllLossBackward>)
